<a href="https://colab.research.google.com/github/DjSteker/huggingface_Tools/blob/main/Plantilla_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalar

In [ ]:
pip install transformers
pip install datasets
pip install evaluate
pip install accelerate -U

In [ ]:
!pip install transformers evaluate datasets accelerate -q

___________________

###Inicializar

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model = "openai-community/gpt2" # https://huggingface.co/openai-community/gpt2


In [ ]:
model="bert-base-uncased" #https://www.philschmid.de/pre-training-bert-habana


In [ ]:
model="tloen/alpaca-lora-7b" # https://huggingface.co/tloen/alpaca-lora-7b


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

____________________________

#Instancias

In [ ]:
from datasets import list_datasets, load_dataset, DatasetInfo
from huggingface_hub import list_datasets, dataset_info
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import  BertConfig, BertModel
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from torch.utils.data import DataLoader, Dataset
from datasets import list_datasets, load_dataset, DatasetInfo
from transformers import Trainer, TrainingArguments


#Instancias Separadas

In [ ]:
from transformers import pipeline

In [ ]:
from huggingface_hub import list_datasets, dataset_info

In [ ]:
from datasets import list_datasets, load_dataset, DatasetInfo

In [ ]:
from transformers import AutoTokenizer

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
import numpy as np
import evaluate

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
from transformers import  BertConfig, BertModel

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from torch.utils.data import DataLoader, Dataset
from datasets import list_datasets, load_dataset, DatasetInfo

In [ ]:
from transformers import Trainer, TrainingArguments


_________________

#Cargar dataset

In [ ]:
all_datasets = list_datasets()
print(f"Existen {len(all_datasets)} datasets")
all_datasets = list_datasets(sort="downloads", direction=-1, limit=5)

In [ ]:
dataset = load_dataset("ecastera/filosofia-es")
dataset

In [ ]:
dataset = load_dataset("yelp_review_full")
dataset

In [ ]:
dataset = load_dataset("hackathon-somos-nlp-2023/winogrande_train_s_spanish")
dataset

In [ ]:
dataset = load_dataset("huggingface/CodeBERTa-small-v1")
dataset

In [ ]:
dataset = load_dataset("matlok/multimodal-python-copilot-training-overview")
dataset

In [ ]:
dataset = load_dataset("hackathon-somos-nlp-2023/Habilidades_Agente_v1")
dataset

In [ ]:
el_quijote = load_dataset("text", data_files="https://mymldatasets.s3.eu-de.cloud-object-storage.appdomain.cloud/el_quijote.txt")
el_quijote
el_quijote.set_format(type='pandas')
df = el_quijote['train'][:]
df.head()


In [ ]:
df = dataset

__________

In [ ]:
atributos = dataset.column_names

# Imprimir los atributos
print(atributos)

__________

In [ ]:
dataset.save_to_disk("./home/winogrande_train_s_spanish1")

In [ ]:
dataset.push_to_hub("DjSteker/")

___________________________

Preparar

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(modelo)

In [ ]:
save_directory = "saved"
tokenizer.save_pretrained(save_directory)

_____

In [ ]:
df = el_quijote['train'][:]
df['name'].value_counts().plot(kind='barh')
plt.show()

___________________

rellenar

In [ ]:
# para un mejor entrenamiento utilizar más datos
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(18119))
#small_train_dataset = dataset["train"].shuffle(seed=42).select(range(51941))
#small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(25971))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(25971))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(modelo)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(modelo, num_labels=5)

In [ ]:
metric = evaluate.load("accuracy")
# calculamos el accuracy de las predicciones
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# definimos los argumentos del entrenamiento
training_args = TrainingArguments(
    'mi-super-modelo',
    evaluation_strategy="steps",
    logging_steps=5,
    num_train_epochs=1,
    push_to_hub=True,
)
# instanciamos el objeto Trainer con todo lo que hemos preparado
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
# 8 min con 80 ejemplos (aprox)
trainer.train()

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments("test-trainer")

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.traing()

#Entrenador 0

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

modelo = "DjSteker/mi-super-modelo"

tokenizer = AutoTokenizer.from_pretrained(modelo)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)


model = AutoModelForSequenceClassification.from_pretrained(modelo, num_labels=5)

metric = evaluate.load("accuracy")

# calculamos el accuracy de las predicciones
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# definimos los argumentos del entrenamiento
training_args = TrainingArguments(
    'mi-super-modelo',
    evaluation_strategy="steps",
    logging_steps=5,
    num_train_epochs=1,
    push_to_hub=True,
)
# instanciamos el objeto Trainer con todo lo que hemos preparado
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

#Entrenador 1

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from torch.utils.data import DataLoader, Dataset
from datasets import list_datasets, load_dataset, DatasetInfo




# Cargar el dataset
el_quijote = load_dataset("text", data_files="https://mymldatasets.s3.eu-de.cloud-object-storage.appdomain.cloud/el_quijote.txt")
train_data = el_quijote['train']


el_quijote.set_format(type='pandas')
df_data = dataset['train'][:] #el_quijote['train'][:]
#df_data.head()

# Preprocesar el texto
tokenizer =BertTokenizer.from_pretrained(model) #BertTokenizer.from_pretrained('DjSteker/mi-super-modelo2')
max_length = 256

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]['text']
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True
        )
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
        }

# Crear los dataloaders
batch_size = 8
train_dataset = CustomDataset(df_data, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Configurar el dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Crear el modelo y el optimizador
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

# Entrenamiento del modelo
num_epochs = 10
total_steps = len(train_loader) * num_epochs

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}')

_____

Guardar modelo

In [ ]:
trainer.save_model("./modelo1")

In [ ]:
trainer.push_to_hub()

In [ ]:
tokenizer.push_to_hub("DjSteker/mi-super-modelo")

In [ ]:
dataset.push_to_hub("DjSteker/yelp_review_full1")

In [ ]:
dataset.save_to_disk(yelp_review_full1)

_________________________

#Jugar

In [ ]:
from transformers import pipeline

#summarizer = pipeline("summarization", model="sshleifer/distibart-cnn-12-6")
summarizer = pipeline("summarization", model)

text = "Javier, un hombre con una camisa a cuadros y calcetines de colores, lucha con un cuadro torcido en la pared. Carlos, su amigo, entra con una caja de pizza en la mano."

print(summarizer(text, max_length=130, min_length=30, dosample=False))

In [ ]:
summarizer = generator("hola ¿ cómo te llamas ?" , max_length=30, num_return_sequences=4,)

print(res)